In [1]:
!pip install arviz -q
!pip install theano-pymc -q
!pip install pymc3==3.10.0 -q

In [2]:
import numpy as np
import scipy as sp
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt
import scipy.stats as stats
import arviz as az

print('Running on PyMC3 v{}'.format(pm.__version__))
print('Running on ArviZ v{}'.format(az.__version__))

Running on PyMC3 v3.10.0
Running on ArviZ v0.11.1


In [3]:
# defining colors for the diferent entities
TCR_color = np.array([0.0, 0.6, 0.0])
CD45_color = np.array([1.0, 0.0, 0.0])
LCK_color = np.array([0.0, 0.0, 0.4])
aLCK_color = np.array([1.0, 0.6, 1.0])
pTCR_color = np.array([1.0, 0.6, 0.0])
membrane_color = 0.6*np.array([1.0, 1.0, 1.0])

In [4]:
# temp
!dir

sample_data


## Metamodel containing all surrogate models and their coupling

### Abbreviations:
ac - slope of 'c' (e.g. at for slope of t)

In [21]:
# Model 1 constants: ##########################
# w_TCR:
# intercept with origin:
mu_rv_b_w_TCR_KS1 = 386.360
sd_rv_b_w_TCR_KS1 = 4.043
# surface t slope:
mu_rv_at_w_TCR_KS1 = 0.102
sd_rv_at_w_TCR_KS1 = 0.060
# surface k slope:
mu_rv_ak_w_TCR_KS1 = 0.016
sd_rv_ak_w_TCR_KS1 = 0.016
# noise:
mu_rv_noise_w_TCR_KS1 = 20.782
sd_rv_noise_w_TCR_KS1 = 1.529

# w_CD45:
# intercept with origin:
mu_rv_b_w_CD45_KS1 = 347.252	
sd_rv_b_w_CD45_KS1= 27.409
# surface t slope:
mu_rv_at_w_CD45_KS1 = 0.124
sd_rv_at_w_CD45_KS1 = 0.118
# surface k slope:
mu_rv_ak_w_CD45_KS1 = 0.270
sd_rv_ak_w_CD45_KS1 = 0.243
# noise:
mu_rv_noise_w_CD45_KS1 = 239.808	
sd_rv_noise_w_CD45_KS1 = 14.057

# dep:
# intercept with origin:
mu_rv_b_dep_KS1 = 44.085
sd_rv_b_dep_KS1 = 4.107
# surface t slope:
mu_rv_at_dep_KS1 = 1.240	
sd_rv_at_dep_KS1 = 0.049
# surface k slope:
mu_rv_ak_dep_KS1 = 0.622	
sd_rv_ak_dep_KS1 = 0.049
# noise:
mu_rv_noise_dep_KS1 = 13.843	
sd_rv_noise_dep_KS1 = 1.032	

# Model 2 constants: ##########################
# lambda (nm)
# sigmoid center Diff:
mu_rv_lambda_aLCK_log_Diff_center_LA2 = -2.577
sd_rv_lambda_aLCK_log_Diff_center_LA2 = 0.472
# sigmoid center Poff:
mu_rv_lambda_aLCK_log_Poff_center_LA2 = -0.674
sd_rv_lambda_aLCK_log_Poff_center_LA2 = 0.328
# surface min:
mu_rv_lambda_aLCK_min_LA2 = 0.003
sd_rv_lambda_aLCK_min_LA2 = 0.004
# surface max:
mu_rv_lambda_aLCK_max_LA2 = 0.492
sd_rv_lambda_aLCK_max_LA2 = 0.135
# sigmoid devisor Diff:
mu_rv_lambda_aLCK_log_Diff_divisor_LA2 = -1.162
sd_rv_lambda_aLCK_log_Diff_divisor_LA2 = 0.222
# sigmoid devisor Poff:
mu_rv_lambda_aLCK_log_Poff_divisor_LA2 = 0.507
sd_rv_lambda_aLCK_log_Poff_divisor_LA2 = 0.150
# Noise
mu_noise_lambda_aLCK_LA2 = 0.055
sd_noise_lambda_aLCK_LA2 = 0.003

# Model 3 constants: ##########################
# gaussian sigma decay_length
mu_rv_decay_length_sigma_TP3 = 61.496
sd_rv_decay_length_sigma_TP3 = 4.399
# gaussian mu decay_length
mu_rv_decay_length_mu_TP3 = -21.977
sd_rv_decay_length_mu_TP3 = 6.779
# gaussian scale decay_length
mu_rv_decay_length_scale_TP3 = 1.023
sd_rv_decay_length_scale_TP3 = 0.147
# gaussian sigma depletion
mu_rv_depletion_sigma_TP3 = 54.601
sd_rv_depletion_sigma_TP3 = 5.185
# gaussian mu depletion
mu_rv_depletion_mu_TP3 = -15.848
sd_rv_depletion_mu_TP3 = 7.019
# gaussian scale depletion
mu_rv_depletion_scale_TP3 = 1.030
sd_rv_depletion_scale_TP3 = 0.148
# rTCR_max_diff noise
mu_rv_noise_rTCR_max_diff_TP3 = 0.048
sd_rv_noise_rTCR_max_diff_TP3 = 0.003

###############################################

def get_metamodel(observed_t_KS1 = None,
                  observed_k_KS1 = None,
                  observed_log_Poff_LA2 = None, 
                  observed_log_Diff_LA2 = None,
                  observed_depletion_TP3 = None,
                  observed_decay_length_TP3 = None):
    
    ''' return a metamodel with all surrogate models '''
    metamodel = pm.Model()
    with metamodel:
        ### model1 - KS (kinetic segregation) ###########################    
        # param_t = pm.Uniform('param_t', 0, 100, observed=observed_t_KS1)
        # param_k = pm.Uniform('param_k', 0, 50, observed=observed_k_KS1)
        rv_t = pm.Uniform('param_t', 0, 100, observed=observed_t_KS1)
        rv_k = pm.Uniform('param_k', 0, 50, observed=observed_k_KS1)

        # w_TCR_KS1 ###########
        rv_at_w_TCR_KS1 = pm.HalfNormal('rv_at_w_TCR_KS1',
                                         sd = mu_rv_at_w_TCR_KS1) # surface t slope
        rv_ak_w_TCR_KS1 = pm.HalfNormal('rv_ak_w_TCR_KS1', 
                                         sd = mu_rv_ak_w_TCR_KS1) # surface k slope
        rv_b_w_TCR_KS1 = pm.Normal('rv_b_w_TCR_KS1', 
                                    mu = mu_rv_b_w_TCR_KS1, 
                                    sd = sd_rv_b_w_TCR_KS1) # surface intercept
        rv_noise_w_TCR_KS1 = pm.HalfNormal('rv_noise_w_TCR_KS1', 
                                            sd = mu_rv_noise_w_TCR_KS1) # noise 
        rv_w_TCR_KS1 = pm.Normal('rv_w_TCR_KS1',
                                  mu = rv_b_w_TCR_KS1 +\
                                  rv_at_w_TCR_KS1*rv_t + rv_ak_w_TCR_KS1*rv_k,
                                  sd = rv_noise_w_TCR_KS1) #

        # w_CD45_KS1 ###########
        rv_at_w_CD45_KS1 = pm.HalfNormal('rv_at_w_CD45_KS1',
                                         sd = mu_rv_at_w_CD45_KS1) # surface t slope
        rv_ak_w_CD45_KS1 = pm.HalfNormal('rv_ak_w_CD45_KS1', 
                                         sd = mu_rv_ak_w_CD45_KS1) # surface k slope
        rv_b_w_CD45_KS1 = pm.Normal('rv_b_w_CD45_KS1', 
                                    mu = mu_rv_b_w_CD45_KS1, 
                                    sd = sd_rv_b_w_CD45_KS1) # surface intercept
        rv_noise_w_CD45_KS1 = pm.HalfNormal('rv_noise_w_CD45_KS1', 
                                            sd = mu_rv_noise_w_CD45_KS1) # noise 
        rv_w_CD45_KS1 = pm.Normal('rv_w_CD45_KS1',
                                  mu = rv_b_w_CD45_KS1 +\
                                  rv_at_w_CD45_KS1*rv_t + rv_ak_w_CD45_KS1*rv_k,
                                  sd = rv_noise_w_CD45_KS1) #

        # dep_KS1 ###########
        rv_at_dep_KS1 = pm.HalfNormal('rv_at_dep_KS1',
                                         sd = mu_rv_at_dep_KS1) # surface t slope
        rv_ak_dep_KS1 = pm.HalfNormal('rv_ak_dep_KS1', 
                                         sd = mu_rv_ak_dep_KS1) # surface k slope
        rv_b_dep_KS1 = pm.Normal('rv_b_dep_KS1', 
                                    mu = mu_rv_b_dep_KS1, 
                                    sd = sd_rv_b_dep_KS1) # surface intercept
        rv_noise_dep_KS1 = pm.HalfNormal('rv_noise_dep_KS1', 
                                            sd = mu_rv_noise_dep_KS1) # noise 
        rv_dep_KS1 = pm.Normal('rv_dep_KS1',
                                  mu = rv_b_dep_KS1 +\
                                  rv_at_dep_KS1*rv_t + rv_ak_dep_KS1*rv_k,
                                  sd = rv_noise_dep_KS1) #

        ### model2 - LA (LCK activation) ###########################    
        # param_log_Diff = pm.Uniform('param_log_Diff', -3, 0, observed= observed_log_Diff_LA2)
        # param_log_Poff = pm.Uniform('param_log_Poff', -5, 0, observed= observed_log_Poff_LA2)
        rv_log_Diff = pm.Uniform('param_log_Diff', -3, 0, observed= observed_log_Diff_LA2)
        rv_log_Poff = pm.Uniform('param_log_Poff', -5, 0, observed= observed_log_Poff_LA2)

        ### lambda aLCK
        noise_lambda_aLCK_LA2 = pm.HalfNormal('noise_lambda_aLCK_LA2',
                                             sd = mu_noise_lambda_aLCK_LA2) # noise 
        rv_lambda_aLCK_min_LA2 = pm.TruncatedNormal('rv_lambda_aLCK_min_LA2',
                                                   mu = mu_rv_lambda_aLCK_min_LA2,
                                                   sd = sd_rv_lambda_aLCK_min_LA2,
                                                   upper = 0.01)
        rv_lambda_aLCK_max_LA2 = pm.TruncatedNormal('rv_lambda_aLCK_max_LA2',
                                                   mu = mu_rv_lambda_aLCK_max_LA2,
                                                   sd = sd_rv_lambda_aLCK_max_LA2,
                                                   lower = 0.01)
        rv_lambda_aLCK_log_Diff_center_LA2 = pm.Normal('rv_lambda_aLCK_log_Diff_center_LA2',
                                                      mu = mu_rv_lambda_aLCK_log_Diff_center_LA2, # 0
                                                      sd = sd_rv_lambda_aLCK_log_Diff_center_LA2) # 1
        rv_lambda_aLCK_log_Diff_divisor_LA2 = pm.TruncatedNormal('rv_lambda_aLCK_log_Diff_divisor_LA2',
                                                                mu = mu_rv_lambda_aLCK_log_Diff_divisor_LA2, 
                                                                sd = sd_rv_lambda_aLCK_log_Diff_divisor_LA2,
                                                                upper = 0)
        rv_lambda_aLCK_log_Poff_center_LA2 = pm.Normal('rv_lambda_aLCK_log_Poff_center_LA2',
                                                       mu = mu_rv_lambda_aLCK_log_Poff_center_LA2, # -3
                                                       sd = sd_rv_lambda_aLCK_log_Poff_center_LA2) # 0.5
        rv_lambda_aLCK_log_Poff_divisor_LA2 = pm.TruncatedNormal('rv_lambda_aLCK_log_Poff_divisor_LA2',
                                                                mu = mu_rv_lambda_aLCK_log_Poff_divisor_LA2, #
                                                                sd = sd_rv_lambda_aLCK_log_Poff_divisor_LA2,
                                                                lower = 0) # 

        rv_tmp_x1 = (rv_log_Diff - rv_lambda_aLCK_log_Diff_center_LA2) / rv_lambda_aLCK_log_Diff_divisor_LA2
        rv_tmp_sig1 = 1.0 / (1 + np.exp(-rv_tmp_x1))
        rv_tmp_x2 = (rv_log_Poff - rv_lambda_aLCK_log_Poff_center_LA2) / rv_lambda_aLCK_log_Poff_divisor_LA2
        rv_tmp_sig2 = 1.0 / (1 + np.exp(-rv_tmp_x2))

        rv_lambda_aLCK_LA2 = pm.Normal('rv_lambda_aLCK_LA', mu=rv_lambda_aLCK_min_LA2 +\
                                  (rv_lambda_aLCK_max_LA2 - rv_lambda_aLCK_min_LA2)*\
                                  rv_tmp_sig1 * rv_tmp_sig2,
                                  sd=noise_lambda_aLCK_LA2) # , observed=lambda_aLCK_obs

        ### Coupling model ########################################
        # from model1:
        rv_w_TCR_C = pm.Normal('rv_w_TCR_C', 
                                   mu = rv_w_TCR_KS1, 
                                   sd = 20) 
        
        rv_w_CD45_C = pm.Normal('rv_w_CD45_C', 
                                   mu = rv_w_CD45_KS1, 
                                   sd = 20) 
           
        rv_dep_C = pm.Normal('rv_dep_C', 
                                   mu = rv_dep_KS1, 
                                   sd = 50) 
        
        # from model2:
        # rv_lambda_aLCK_C = pm.Normal('rv_lambda_aLCK_C', 
        #                            mu = rv_lambda_aLCK_LA2, 
        #                            sd = 2)

        rv_deday_length_aLCK_C = pm.Normal('rv_deday_length_aLCK_C', 
                                   mu = 1/rv_lambda_aLCK_LA2, 
                                   sd = 50) 
        
        ### Model 3 (TCR phosphorylation) ################
        rv_decay_length_TP = pm.Uniform('rv_decay_length_TP', 0, 300,
                                        observed=observed_decay_length_TP3)
        rv_depletion_TP = pm.Uniform('rv_depletion_TP', 0, 800,
                                     observed=observed_depletion_TP3) 


        ### rTCR_max_diff
        rv_noise_rTCR_max_diff_TP3 = pm.HalfNormal('rv_noise_rTCR_max_diff_TP3',
                                              sd=mu_rv_noise_rTCR_max_diff_TP3) # noise 
        
        ### decay_length:
        rv_decay_length_sigma_TP3 = pm.Normal('rv_decay_length_sigma_TP3',
                                             mu=mu_rv_decay_length_sigma_TP3,
                                             sd=sd_rv_decay_length_sigma_TP3)
        rv_decay_length_mu_TP3 = pm.Normal('rv_decay_length_mu_TP3',
                                             mu=mu_rv_decay_length_mu_TP3,
                                             sd=sd_rv_decay_length_mu_TP3)
        rv_decay_length_scale_TP3 = pm.Normal('rv_decay_length_scale_TP3',
                                             mu=mu_rv_decay_length_scale_TP3,
                                             sd=sd_rv_decay_length_scale_TP3)
        
        rv_decay_length_gaussian_TP3 = rv_decay_length_scale_TP3*\
            np.exp(-0.5*((rv_decay_length_TP - rv_decay_length_mu_TP3)/rv_decay_length_sigma_TP3)**2)
        
        ### depletion:
        rv_depletion_sigma_TP3 = pm.Normal('rv_depletion_sigma_TP3',
                                             mu=mu_rv_depletion_sigma_TP3,
                                             sd=sd_rv_depletion_sigma_TP3)
        rv_depletion_mu_TP3 = pm.Normal('rv_depletion_mu_TP3',
                                             mu=mu_rv_depletion_mu_TP3,
                                             sd=sd_rv_depletion_mu_TP3)
        rv_depletion_scale_TP3 = pm.Normal('rv_depletion_scale_TP3',
                                             mu=mu_rv_depletion_scale_TP3,
                                             sd=sd_rv_depletion_scale_TP3)
        
        rv_depletion_gaussian_TP3 = rv_depletion_scale_TP3*\
            np.exp(-0.5*((rv_depletion_TP - rv_depletion_mu_TP3)/rv_depletion_sigma_TP3)**2)       
        
        rv_rTCR_max_diff_TP3 = pm.Normal('rv_rTCR_max_diff_TP3', mu=rv_decay_length_gaussian_TP3*\
                                    rv_depletion_gaussian_TP3,                     
                                    sd=rv_noise_rTCR_max_diff_TP3,
                                    observed=rTCR_max_diff_obs)

    return metamodel


# Down direction
metamodel= get_metamodel(observed_t_KS1= 75.0,
                         observed_k_KS1= 25.0,
                         observed_log_Poff_LA2= -3.0,
                         observed_log_Diff_LA2= -2.0,
                         observed_depletion_TP3= 200, # None
                         observed_decay_length_TP3= 100) # None

"""
# Up direction
# metamodel= get_metamodel(observed_t_KS1= None, # 75.0, None
#                          observed_k_KS1= None, # 34.0, # 
#                          observed_log_Poff_LA2= None, # -3.0, # None
#                          observed_log_Diff_LA2= None, # 1.0, # None
#                          observed_dw_pTCR_TP3= 50, # None, # 50
#                          observed_dm_pTCR_TP3= 600) # None # 600

gv_metamodel = pm.model_to_graphviz(metamodel)
# display(gv_metamodel)
"""
TCR_color

NameError: ignored

In [15]:
gv_metamodel = pm.model_to_graphviz(metamodel)
# display(gv_metamodel)

NameError: ignored

In [ ]:
from google.colab import files
gv_metamodel.render("metamodel_graph", format="png")
files.download("metamodel_graph.png") # Download locally from colab

In [ ]:
with metamodel:
    trace_metamodel = pm.sample(2000, chains=4)

In [ ]:
pm.traceplot(trace_metamodel);

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
pm.summary(trace_metamodel).round(2)

In [ ]:
# vars(metamodel)

In [ ]:
pm.summary(trace_metamodel, ['rv_dm_pTCR_TP2', 'rv_dw_pTCR_TP2'])

In [ ]:
# metamodel= get_metamodel(observed_t_KS1= 75.0,
#                          observed_k_KS1= 34.0,
#                          observed_log_Poff_LA3= -3.0,
#                          observed_log_Diff_LA3= 1.0
# model1:

# model2:

# model3:

mu_rv_dm_pTCR_TP2 = trace_metamodel.rv_dm_pTCR_TP2.mean()
sd_rv_dm_pTCR_TP2 = trace_metamodel.rv_dm_pTCR_TP2.std()
mu_rv_dw_pTCR_TP2 = trace_metamodel.rv_dw_pTCR_TP2.mean()
sd_rv_dw_pTCR_TP2 = trace_metamodel.rv_dw_pTCR_TP2.std()

print('mu_rv_dm_pTCR_TP2 =', mu_rv_dm_pTCR_TP2)
print('sd_rv_dm_pTCR_TP2 =', sd_rv_dm_pTCR_TP2)
print('mu_rv_dw_pTCR_TP2 =', mu_rv_dw_pTCR_TP2)
print('sd_rv_dw_pTCR_TP2 =', sd_rv_dw_pTCR_TP2)

In [ ]:
xx = np.linspace(0, 1000, 201)

# TCR:
d_TCR = np.zeros(xx.shape)
d_TCR[xx < 250] = 1
# CD45:
mu_rv_dw_aLCK_TP2 = trace_metamodel.rv_dw_aLCK_TP2.mean()	
mu_rv_dm_CD45_TP2 = trace_metamodel.rv_dm_CD45_TP2.mean()
d_CD45 = np.zeros(xx.shape)
d_CD45[(xx > mu_rv_dm_CD45_TP2-250) & (xx < mu_rv_dm_CD45_TP2+250)] = 1
# aLCK:
d_aLCK = 2*np.exp(-0.5*((xx - mu_rv_dm_CD45_TP2)/mu_rv_dw_aLCK_TP2)**2)
# pTCR:
d_pTCR = np.exp(-0.5*((xx - mu_rv_dm_pTCR_TP2)/mu_rv_dw_pTCR_TP2)**2)


fig, ax3=plt.subplots(figsize=[12,3])

ax3.plot(xx, d_TCR, '-', color=TCR_color)
ax3.plot(xx, d_CD45, '-', color=CD45_color)
ax3.plot(xx, d_aLCK, '-', color=aLCK_color)
ax3.plot(xx, d_pTCR, '-', color=pTCR_color)

In [ ]:
plt.scatter(trace_metamodel.rv_dm_CD45_TP2, trace_metamodel.rv_dw_aLCK_TP2, s=1)
plt.xlabel('rv_dm_CD45_TP2(nm)')
plt.ylabel('rv_dw_aLCK_TP2(nm)')
plt.xlim(300, 1000)